# Import and Load Data

In [38]:
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from IPython.core.debugger import set_trace
from pathlib import Path

# Pytorch
import torch
from torch import nn
from torch.nn import functional as F
from torch import Tensor
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
from torch.nn.modules.loss import CrossEntropyLoss


from one_cycle import OneCycle, update_lr, update_mom

In [2]:
working_dir = Path()
normal_path = working_dir / 'Data' / 'Normal'
DE_path = working_dir / 'Data' / '12k_DE'
FE_path = working_dir / 'Data' / '12k_FE'

In [3]:
# Load the data which is in matlab format into dic for further preprocessing
normal_data = {}
for i, filepath in enumerate(normal_path.glob('*.mat')):
    key_name = str(filepath).split('\\')[-1]    #strip the folder path and get the filename only.
    normal_data[key_name] = scipy.io.loadmat(filepath)

DE_data = {}
for i, filepath in enumerate(DE_path.glob('*.mat')):
    key_name = str(filepath).split('\\')[-1]    #strip the folder path and get the filename only.
    DE_data[key_name] = scipy.io.loadmat(filepath)


In [4]:
# Remove redundant information of the matlab data.
for key, values in normal_data.items():
    del values['__header__']
    del values['__version__']    
    del values['__globals__']    
    
for key, values in DE_data.items():
    del values['__header__']
    del values['__version__']    
    del values['__globals__']

In [5]:
for k1,v1 in DE_data.items():
    del_list = []
    for k2,v2 in list(v1.items()):
        if 'DE_time' in k2:
            v1['DE_time'] = v1.pop(k2)
        elif 'BA_time' in k2:
            v1['BA_time'] = v1.pop(k2)
        elif 'FE_time' in k2:
            v1['FE_time'] = v1.pop(k2)
        elif 'RPM' in k2:
            v1['RPM'] = v1.pop(k2)

In [6]:
for k1,v1 in normal_data.items():
    del_list = []
    for k2,v2 in list(v1.items()):
        if 'DE_time' in k2:
            v1['DE_time'] = v1.pop(k2)
        elif 'BA_time' in k2:
            v1['BA_time'] = v1.pop(k2)
        elif 'FE_time' in k2:
            v1['FE_time'] = v1.pop(k2)
        elif 'RPM' in k2:
            v1['RPM'] = v1.pop(k2)

# Functions

In [7]:
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )

def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(f'EPOCH {epoch}: \t', val_loss)
        
    return model

In [8]:
def fit2(epochs, model, loss_func, opt, train_dl, valid_dl):
    print('EPOCH', '\t', 'Val Loss', '\t', 'Accuracy')
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            loss = [loss_func(model(xb), yb) for xb, yb in valid_dl]
            loss = torch.stack(loss, dim=0).numpy()
            predictions = [torch.argmax(model(xb), dim=1) for xb, yb in valid_dl]
#             set_trace()
            predictions = torch.cat(predictions, dim=0).numpy()
#         set_trace()
        val_loss = np.mean(loss)
        accuracy = np.mean((predictions == valid_dl.dataset.tensors[1].numpy()))

        print(f'{epoch}: \t', f'{val_loss:.05f}', '\t', f'{accuracy:.05f}')
        
    return model

In [9]:
def fit_one_cycle(epochs, model, loss_func, opt, train_dl, valid_dl, one_cycle_scheduler):
    print('EPOCH', '\t', 'Val Loss', '\t', 'Accuracy')
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)
            lr, mom = onecycle.calc()
            update_lr(opt, lr)
            update_mom(opt, mom)

        model.eval()
        with torch.no_grad():
            loss = [loss_func(model(xb), yb) for xb, yb in valid_dl]
            loss = torch.stack(loss, dim=0).numpy()
            predictions = [torch.argmax(model(xb), dim=1) for xb, yb in valid_dl]
#             set_trace()
            predictions = torch.cat(predictions, dim=0).numpy()
#         set_trace()
        val_loss = np.mean(loss)
        accuracy = np.mean((predictions == valid_dl.dataset.tensors[1].numpy()))

        print(f'{epoch}: \t', f'{val_loss:.05f}', '\t', f'{accuracy:.05f}')
        
    return model

# Preprocess

In [10]:
df_BE = pd.DataFrame.from_dict(DE_data).T

In [11]:
df_Normal = pd.DataFrame.from_dict(normal_data).T

In [12]:
def label(x):
    if 'B' in x:
        return 'B'
    elif 'IR' in x:
        return 'IR'
    elif 'OR' in x:
        return 'OR'

In [13]:
df_Normal = df_Normal.reset_index()
df_Normal['label'] = 'N'

In [14]:
df_BE = df_BE.reset_index()
df_BE['label'] = df_BE['index'].apply(label)

In [15]:
df_Normal = df_Normal.drop(['FE_time', 'RPM', 'ans'], axis=1)
df_BE = df_BE.drop(['FE_time','BA_time', 'RPM'], axis=1)

In [16]:
df_Normal.head()

,index,DE_time,label
0,Normal_0.mat,"[[0.05319692307692307], [0.08866153846153846],...",N
1,Normal_1.mat,"[[0.046104], [-0.03713353846153846], [-0.08949...",N
2,Normal_2.mat,"[[0.06425353846153846], [0.06300184615384616],...",N
3,Normal_3.mat,"[[0.014603076923076923], [0.05444861538461539]...",N


In [17]:
df_BE.sample(5)

,index,DE_time,label
41,OR007@6_1.mat,"[[-0.1104558882235529], [0.21563263473053892],...",OR
5,B014_1.mat,"[[0.005522794411177645], [-0.10379604790419161...",B
40,OR007@6_0.mat,"[[0.008527844311377245], [0.4235496007984032],...",OR
57,OR021@6_1.mat,"[[-0.025989620758483035], [-0.0272078842315369...",OR
11,B021_3.mat,"[[0.07244606786427145], [0.04791836327345309],...",B


In [18]:
segment_length = 2048
normal_dic = {}
idx = 0
for i in range(df_Normal.shape[0]):
    n = len(df_Normal.iloc[i,1])
    num_segments = n // segment_length
    for segment in range(num_segments):
        normal_dic[idx] = {
            'signal': df_Normal.iloc[i,1][segment*segment_length:segment*segment_length+segment_length],
            'label': df_Normal.iloc[i,2],
            'filename' : df_Normal.iloc[i,0]
                          }
        idx += 1

In [19]:
faulty_dic = {}
idx = 0
for i in range(df_BE.shape[0]):
    n = len(df_BE.iloc[i,1])
    num_segments = n // segment_length
    for segment in range(num_segments):
        faulty_dic[idx] = {
            'signal': df_BE.iloc[i,1][segment*segment_length:segment*segment_length+segment_length],
            'label': df_BE.iloc[i,2],
            'filename' : df_BE.iloc[i,0]
                          }
        idx += 1

In [20]:
df_normal_tmp = pd.DataFrame.from_dict(normal_dic,orient='index')
df_faulty_tmp = pd.DataFrame.from_dict(faulty_dic,orient='index')

In [21]:
df_normal_processed = pd.concat([df_normal_tmp, pd.DataFrame(np.hstack(df_normal_tmp["signal"].values).T)], axis=1 )
df_faulty_processed = pd.concat([df_faulty_tmp, pd.DataFrame(np.hstack(df_faulty_tmp["signal"].values).T)], axis=1 )

In [22]:
df_all = pd.concat([df_normal_processed, df_faulty_processed], axis=0, ignore_index=True).drop(['signal'],axis=1)

In [23]:
map_label = {'N':0, 'B':1, 'IR':2, 'OR':3}
df_all['label'] = df_all['label'].map(map_label)

In [24]:
df_all.label.unique()

array([0, 1, 2, 3], dtype=int64)

In [25]:
features = df_all.columns[2:]
target = 'label'
print(len(features))

2048


# 1D CNN

In [ ]:
# seg_length 1000, lowest loss 0.011
class CNN_1D(nn.Module):
    def __init__(self, n_in):
        super().__init__()
        self.n_in = n_in
        self.conv_in = nn.Conv1d(1, 32, (9,), stride=1, padding=4)
        self.conv_1 = nn.Conv1d(32, 64, (5,), stride=1, padding=2)
        self.conv_2 = nn.Conv1d(64, 128, (5,), stride=1, padding=2)
        self.maxpool = nn.MaxPool1d(2,stride=2)
        self.averagepool = nn.AvgPool1d(2,stride=2)
        self.dropout = nn.Dropout(p=0.5)
        self.linear1 = nn.Linear(self.n_in*64 //4, 50)
        self.linear2 = nn.Linear(50, 4)

        
    def forward(self, x):
        x = x.view(-1, 1, self.n_in)
        x = F.relu(self.conv_in(x))
        x = self.maxpool(x)
        x = F.relu(self.conv_1(x))
        x = self.averagepool(x)
        x = x.view(-1, self.n_in*64//4)
        x = self.linear1(x)
        return self.linear2(x)
    

In [46]:
# seg_length 1000, lowest loss 0.014
class CNN_1D(nn.Module):
    def __init__(self, n_in):
        super().__init__()
        self.n_in = n_in
        self.conv_in = nn.Conv1d(1, 32, (9,), stride=1, padding=4)
        self.conv_1 = nn.Conv1d(32, 64, (5,), stride=1, padding=2)
        self.conv_2 = nn.Conv1d(64, 128, (5,), stride=1, padding=2)
        self.maxpool = nn.MaxPool1d(2,stride=2)
        self.averagepool = nn.AvgPool1d(2,stride=2)
        self.dropout = nn.Dropout(p=0.5)
        self.linear1 = nn.Linear(self.n_in*64 //4, 4)

        
    def forward(self, x):
        x = x.view(-1, 1, self.n_in)
        x = F.relu(self.conv_in(x))
        x = self.maxpool(x)
        x = F.relu(self.conv_1(x))
        x = self.averagepool(x)
        x = x.view(-1, self.n_in*64//4)
        return self.linear1(x)
    

In [ ]:
# seg_length 1000, lowest loss 
class CNN_1D(nn.Module):
    def __init__(self, n_in):
        super().__init__()
        self.n_in = n_in
        self.conv_in = nn.Conv1d(1, 32, (9,), stride=1, padding=4)
        self.conv_1 = nn.Conv1d(32, 64, (5,), stride=1, padding=2)
        self.conv_2 = nn.Conv1d(64, 128, (5,), stride=1, padding=2)
        self.maxpool = nn.MaxPool1d(2,stride=2)
        self.averagepool = nn.AvgPool1d(2,stride=2)
        self.dropout = nn.Dropout(p=0.5)
        self.linear1 = nn.Linear(self.n_in*64 //4, 4)

        
    def forward(self, x):
        x = x.view(-1, 1, self.n_in)
        x = self.dropout(F.relu(self.conv_in(x)))
        x = self.maxpool(x)
        x = self.dropout(F.relu(self.conv_1(x)))
        x = self.averagepool(x)
        x = x.view(-1, self.n_in*64//4)
        return self.linear1(x)
    

In [55]:
class CNN_1D(nn.Module):
    def __init__(self, n_in):
        super().__init__()
        self.n_in = n_in
        self.layer1 = nn.Sequential(
            nn.Conv1d(1, 32, (9,), stride=1, padding=4),
            nn.ReLU(),
            nn.MaxPool1d(2,stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv1d(32, 64, (5,), stride=1, padding=2),
            nn.ReLU()
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv1d(64, 128, (5,), stride=1, padding=2),
            nn.ReLU(),
            nn.AvgPool1d(2,stride=2)
        )
        
        self.linear1 = nn.Linear(self.n_in*128 //4, 50)
        self.linear2 = nn.Linear(50, 4)

        
    def forward(self, x):
        x = x.view(-1, 1, self.n_in)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.view(-1, self.n_in*128//4)
        x = self.linear1(x)
        return self.linear2(x)
    

In [ ]:
class CNN_1D(nn.Module):
    def __init__(self, n_in):
        super().__init__()
        self.n_in = n_in
        self.layer1 = nn.Sequential(
            nn.Conv1d(1, 64, (9,), stride=1, padding=4),
            nn.ReLU(),
            nn.MaxPool1d(2,stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv1d(64, 128, (5,), stride=1, padding=2),
            nn.ReLU()
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv1d(128, 128, (5,), stride=1, padding=2),
            nn.ReLU(),
            nn.AvgPool1d(2,stride=2)
        )
        
        self.linear1 = nn.Linear(self.n_in*128 //4, 4)

        
    def forward(self, x):
        x = x.view(-1, 1, self.n_in)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.view(-1, self.n_in*128//4)
        x = self.linear1(x)
        return x
    

In [58]:
class CNN_1D(nn.Module):
    def __init__(self, n_in):
        super().__init__()
        self.n_in = n_in
        self.layer1 = nn.Sequential(
            nn.Conv1d(1, 64, (9,), stride=1, padding=4),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.MaxPool1d(2,stride=2)
        )
        
        
        self.layer2 = nn.Sequential(
            nn.Conv1d(64, 128, (5,), stride=1, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.AvgPool1d(2,stride=2)
        )
        
        self.linear1 = nn.Linear(self.n_in*128 //4, 4)

        
    def forward(self, x):
        x = x.view(-1, 1, self.n_in)
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, self.n_in*128//4)
        return self.linear1(x)
        

In [ ]:
class CNN_1D(nn.Module):
    def __init__(self, n_in):
        super().__init__()
        self.n_in = n_in
        self.layer1 = nn.Sequential(
            nn.Conv1d(1, 64, (9,), stride=1, padding=4),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.MaxPool1d(2,stride=2)
        )
        
        
        self.layer2 = nn.Sequential(
            nn.Conv1d(64, 64, (5,), stride=1, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.MaxPool1d(2,stride=2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv1d(64, 128, (5,), stride=1, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.MaxPool1d(2,stride=2)
        )
        
        self.linear1 = nn.Linear(self.n_in*128 //8, 4)

        
    def forward(self, x):
        x = x.view(-1, 1, self.n_in)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.view(-1, self.n_in*128//8)
        return self.linear1(x)
        

In [ ]:
class CNN_1D(nn.Module):
    def __init__(self, n_in, layers = [1, 64, 64, 128]):
        super().__init__()
        self.n_in = n_in
        self.layers = layers
        for layer_num, layer_size in layers[:-1]:
            setattr(f'layer_{layer_num + 1}') = nn.Sequential(
                nn.Conv1d(layers[layer_num], layers[layer_num + 1], (5,), stride=1, padding=2),
                nn.BatchNorm1d(layers[layer_num + 1]),
                nn.ReLU(),
                nn.Dropout(p=0.5),
                nn.MaxPool1d(2,stride=2)
            )
        
        self.linear1 = nn.Linear(self.n_in*layers[-1] // 2**(len(layers)-1), 4)

        
    def forward(self, x):
        x = x.view(-1, 1, self.n_in)
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, self.n_in*128//8)
        return self.linear1(x)

# Training

In [34]:
X_train, X_valid, y_train, y_valid = train_test_split(df_all[features], 
                                                      df_all[target], 
                                                      test_size=0.20, random_state=42, shuffle=True
                                                     )

In [35]:
lr = 0.01
bs = 64
wd = 1e-3
epochs = 20
loss_func = CrossEntropyLoss()

In [36]:
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_valid = torch.tensor(X_valid.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_valid = torch.tensor(y_valid.values, dtype=torch.long)

train_ds = TensorDataset(X_train, y_train)
valid_ds = TensorDataset(X_valid, y_valid)
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)

In [ ]:
%%time
model = CNN_1D(len(features))
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
# opt = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=wd)
model = fit2(epochs, model, loss_func, opt, train_dl, valid_dl)

In [ ]:
%%time
model = CNN_1D(1000)
# opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
opt = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=wd)
model = fit2(epochs, model, loss_func, opt, train_dl, valid_dl)

In [ ]:
lr = 0.01
bs = 64
wd = 1e-3
epochs = 10
model = fit2(epochs, model, loss_func, opt, train_dl, valid_dl)

In [ ]:
predictions = []
with torch.no_grad():
    model.eval()
    for xb, _ in valid_dl:
        pred = F.softmax(model(xb),dim=1)
        prob, pred = torch.max(pred,1)
#         print(pred.shape)
        predictions.append(pred)
        

In [ ]:
np.mean(y_valid.numpy() == torch.cat(predictions, dim=0).numpy())

In [ ]:
print(np.sum(y_valid.numpy() == torch.cat(predictions, dim=0).numpy()))
print(len(y_valid))

# Fit One Cycle

In [59]:
lr = 0.03
bs = 64
wd = 1e-3
epochs = 10
loss_func = CrossEntropyLoss()
onecycle = OneCycle(int(len(X_train) * epochs / bs), lr, prcnt=(epochs - 82) * 100/epochs, momentum_vals=(0.95, 0.8))

In [60]:
%%time
model = CNN_1D(len(features))
opt = optim.SGD(model.parameters(), lr=lr/10, momentum=0.95, weight_decay=wd)
# opt = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=wd)
model = fit_one_cycle(epochs, model, loss_func, opt, train_dl, valid_dl, onecycle)

EPOCH 	 Val Loss 	 Accuracy
0: 	 2.79584 	 0.35240
1: 	 0.70464 	 0.77460
2: 	 0.36010 	 0.85240
3: 	 0.26214 	 0.88330
4: 	 0.16237 	 0.94050
5: 	 0.48454 	 0.82609
6: 	 0.14245 	 0.94279
7: 	 0.33786 	 0.85126
8: 	 0.10058 	 0.96453
9: 	 0.08943 	 0.96224
Wall time: 10min 57s
